### **Ollama Installation**

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

###**Run Ollama in background parallel**

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama","serve"])

thread=threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

###**Installing Ollama**

In [ ]:
!pip install ollama

### **Pull Embedding Model**

In [ ]:
!ollama pull all-minilm

### **Converting Text into Vector**

In [ ]:
import ollama
#loads ollama library into program

response=ollama.embeddings(
    model="all-minilm",
    prompt="The sky is blue because of Rayleigh Scattering"
)

In [ ]:
print(response)

In [ ]:
print(len(response["embedding"]))

The sentence converted into 384 dimensional vector

### **Vector Databases & Redis Chat Memory (with LangChain + Ollama Embeddings)**

**1.Install packages**

In [ ]:
!pip install langchain
#framework for use LLM and other model
!pip install langchain_community
#tool for using PDF loader, web base loader
!pip install langchain_core
#tool for use prompts,documents,runnable
!pip install langchain-redis
#LangChain uses Redis in a special way — as a memory store for conversations.
#It's like giving chatbot a brain that remembers previous chats,stored in redis
!pip install redis
#Redis is an open-source in-memory database
!pip install qdrant_client
!pip install pinecone-client

**2.Import libraries**

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader

**3.Load web document**

In [ ]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/LangChain")
document=loader.load()
print(document)

**4.Split the document**

In [ ]:
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0
)
chunk_document=splitter.split_documents(document)
print(chunk_document)

In [ ]:
len(chunk_document)

In [ ]:
chunk_document[:5]

## **⚡ FAISS with Ollama Embeddings**

In [ ]:
#inatall faiss packages
!pip install faiss-cpu

In [ ]:
#import libraries both for faiss and ollama
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings

In [ ]:
#create an embedding model
embedding_model=OllamaEmbeddings(model="all-minilm")

In [ ]:
#convert text into vector
vector_store=FAISS.from_documents(chunk_document,embedding_model)

In [ ]:
#save to disk
vector_store.save_local("faiss_index_of_chunk_document")

In [ ]:
#load the document
load_document=FAISS.load_local(
    "faiss_index_of_chunk_document",
    embedding_model,
    allow_dangerous_deserialization=True)


#here embedding model is used to convert the user question into vector


In [ ]:
#metadata filtering
filtered_answer=load_document.similarity_search_with_score(
    "What is langchain?",
    k=3,
    filter={
        "source":["https://en.wikipedia.org/wiki/LangChain"]
    }
)

#here FAISS database store chunk as (page_content= .......,metadata= {source:...... }.......}.It represents a list of items so "source":",.,.,.,.,."
#top 3 matches will be shown and the match which has less score will give priority

In [ ]:
for doc, score in filtered_answer:
    print("Score:", score)
    print("Content:", doc.page_content)
    print("Source:", doc.metadata.get("source"))
    print("-" * 100)

In [ ]:
print(filtered_answer)

In [ ]:
#without metadata filtering
load_document.similarity_search(
    "What is langchain?",
    k=2
)

## `**Hybrid Search ((FAISS + Ollama Embedding)(Semantic Search) + Keyword Search(lexical Search)) **`

**BM25 (Best Matching 25) is a ranking algorithm used to find the most relevant documents based on a user's query — especially in keyword-based search systems like search engines.**

### **BM25 Setup**

In [ ]:
#install packages
!pip install rank_bm25

In [ ]:
#import libraries
from rank_bm25 import BM25Okapi

In [ ]:
#BM25 setup

#1.load page_content from document,extracts only page_content without metadata
texts=[]
for doc in chunk_document:
  text=doc.page_content #get the page content from real document
  texts.append(text)

#2.spilt the text of th document,converts raw text into tokens
tokenized_text=[]
for token in texts:
  token_text=token.split(" ")
  tokenized_text.append(token_text)

#3.build an index and calculate how relevant each document is to a query.
bm25=BM25Okapi(tokenized_text)

In [ ]:
#testing
query="what is langchain?"
tokenized_query=query.split(" ")
bm25_scroes=bm25.get_scores(tokenized_query)
print("Scroes : ", bm25_scroes)

In [ ]:
# find and return the top N most relevant documents based on the BM25 similarity score between your user’s query and the stored document chunks.

# 1. Setting top document
top_n=2

# 2.  Pair each score with its corresponding index
## Example: [(0, 0.0), (1, 0.5), (2, 2.1), (3, 0.0), (4, 3.0), ...]
#>>(index,scores)
indexed_scores=list(enumerate(bm25_scroes))

#3.sorting indexed scores based on scores..
#item[1] iterate the second item cause score is the second item
sorted_indexed_scores=sorted(
    indexed_scores,
    key=lambda item : item[1],
    reverse=True )

#4. Select the top N scored documents
top_documents=sorted_indexed_scores[:top_n]

#5. Now get the document by indecies
bm25_document=[]
for index,score in top_documents:
  doc_index=chunk_document[index]
  bm25_document.append(doc_index)


In [ ]:
faiss_database=vector_store.similarity_search(query,k=2)

In [ ]:
#combine faiss db and bm25 (hybrid search)

all_document=faiss_database+bm25_document
unique_docs_dict = {}
for doc in all_document:
    unique_docs_dict[doc.page_content] = doc

combined_docs = list(unique_docs_dict.values())

In [ ]:
len(combined_docs)

In [ ]:
combined_docs

In [ ]:
!pip install langchain_community

# **ChromaDB**

In [ ]:
#install packages
!pip install chromadb

In [ ]:
#import libraries
from langchain_community.vectorstores import Chroma

In [ ]:
#create an embedding model
embedding_model=OllamaEmbeddings(model="all-minilm")

In [ ]:
#chunk_document
chunk_document

In [ ]:
#create a vector store to convert text into vector and save it into a database
vector_store_chroma=Chroma.from_documents(
    chunk_document,
    embedding_model,
    persist_directory="chroma_store" #save to local disk
)

In [ ]:
#Add new document to chroma_store
vector_store_chroma.add_texts(["Another Document"],metadatas=[{"source":"Manual"}])

Chroma uses this ID to keep track of document.This ID can be used later to delete or update that specific document.basically, it's like a document name created automatically by Chroma.

In [ ]:
#similarity search
vector_store_chroma.similarity_search("What is langchain?",k=2)

In [ ]:
#get the details about an document
vector_store_chroma.get("c672ec27-d1a5-453f-8f0e-08a1a46dd768")

In [ ]:
#Access the chroma database collection
chroma_collection=vector_store_chroma._collection

In [ ]:
print(chroma_collection)

In [ ]:
#Get all IDs of vector_store_chroma database
all_chroma_collection_ids=chroma_collection.get()

print("All ChromaDB Document IDs:",all_chroma_collection_ids)

In [ ]:
#Get all IDs of vector_store_chroma database
all_chroma_collection_ids=chroma_collection.get(include=[])

print("All ChromaDB Document IDs:",all_chroma_collection_ids)

In [ ]:
#Get all IDs of vector_store_chroma database
all_chroma_collection_ids=chroma_collection.get(include=[])['ids']

print("All ChromaDB Document IDs:",all_chroma_collection_ids)

In [ ]:
#delete one item
vector_store_chroma.delete(ids=["2a390620-4633-427e-abe8-2dc61d31b179"])

In [ ]:
all_chroma_collection_ids=chroma_collection.get(include=[])['ids']

print("All ChromaDB Document IDs:",all_chroma_collection_ids)

In [ ]:
vector_store_chroma.get("2a390620-4633-427e-abe8-2dc61d31b179")

In [ ]:
#metadata filtering
filtered_result=vector_store_chroma.similarity_search(
    "what is langchain?",
    k=2,
    filter={
        "source": "https://en.wikipedia.org/wiki/LangChain"
    }
)
print(filtered_result)

In [ ]:
#Hybrid Search
#1.Extract only the page_content from document
texts=[]
for doc in chunk_document:
  text=doc.page_content
  texts.append(text)

print(texts)

In [ ]:
#2.Tokenize the text
tokenized_text=[]
for doc in texts:
  token=doc.split(" ")
  tokenized_text.append(token)

print(tokenized_text)

In [ ]:
#3.bm25 for hybrid search
bm25=BM25Okapi(tokenized_text)
print(bm25)

It's showing the memory address of the object

In [ ]:
#testing
query="What is langchain"

#split the query
splited_query=query.split(" ")
print("Splitted Query:",splited_query)

#get bm25 scores
bm25_scores=bm25.get_scores(splited_query)
print("BM25 Scores:",bm25_scores)

In [ ]:
#get bm25 searched document rank
bm25_rank=enumerate(bm25_scores)
print(bm25_rank)
bm25_rank=sorted(enumerate(bm25_scores))
print(bm25_rank)
bm25_rank=sorted(enumerate(bm25_scores),key= lambda x:x[1])
print(bm25_rank)
bm25_ranks=sorted(enumerate(bm25_scores), key=lambda x:x[1],reverse=True)
print(bm25_ranks)


In [ ]:
#print the top 2 document
top_documents_choroma=bm25_ranks[:2]
print(top_documents)

In [ ]:
#content of the top2 document
bm25_chroma_document=[]
for index,score in top_documents:
  document_index=chunk_document[index]
  bm25_chroma_document.append(document_index)

print(bm25_chroma_document)

In [ ]:
#create semantic seached document
semantic_document_chroma=vector_store_chroma.similarity_search(query,k=5)

In [ ]:
#Combining semantic and keyword based search

all_documents_chroma=semantic_document_chroma+bm25_chroma_document
print(all_documents_chroma)

In [ ]:
hybrid_search_chroma_dict={}
for doc in all_documents_chroma:
  hybrid_search_chroma_dict[doc.page_content]=doc
  print(doc)


print("+"*100)
print(hybrid_search_chroma_dict)

In [ ]:
hybrid_documet_chroma=list(hybrid_search_chroma_dict.values())
print(hybrid_documet_chroma)

In [ ]:
hybrid_documet_chroma